# Multivariable Linear Regression (Train, & Test) Notebook
------------------------------------
Dataset is Automobile_data.txt

This dataset contains prices of cars based on many parameters like make, fuel-type, engine-size...etc.

We will choose more than two parameters in this notebook, so we won't be able to visualize it like previously. However, we would be able to tell how good the model is using Test partition of the dataset.

## Part One: Reading & Preparing Data

In [1]:
# importing needed libraries
import pandas as pd 
import numpy as np

In [2]:
# read dataset from txt file could be read using read_csv function in pandas, but you have to tell it how data is seperated from each other.
# if you opened the txt file, you would see that in each row, data is seperated using a comma
df = pd.read_csv('automobile_data.txt', sep=',')
print(len(df.columns))
df.columns

26


Index(['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration',
       'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
       'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type',
       'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke',
       'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'price'],
      dtype='object')

In [3]:
df[0:5]

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [4]:
# As you saw above, there are some values missing and a '?' is written in its cell.
# This will cause an issue in the training, and we want to remove the rows having missing data
# We will first replace the '?' with Null (None), and then use dropna function in pandas that remove rows having Null values
# You can notice how the size of the dataset decreased after doing this.
print(len(df))
df = df.replace('?',np.NaN)
df = df.dropna()
print(len(df))


205
159


In [5]:
# There is not '?' anymore
df[0:5]

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
6,1,158,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710
8,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875
10,2,192,bmw,gas,std,two,sedan,rwd,front,101.2,...,108,mpfi,3.50,2.80,8.8,101,5800,23,29,16430


In [6]:
df.dtypes

symboling              int64
normalized-losses     object
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                  object
stroke                object
compression-ratio    float64
horsepower            object
peak-rpm              object
city-mpg               int64
highway-mpg            int64
price                 object
dtype: object

There is a problem in the dataset here.

First, some columns which have actual numerical values, have data types of object and not float64
So we will convert these to float64. These columns are: normalized-losses, bore, stroke, horsepower, peak-rpm and price.

Secondly, the columns that have strings .. these columns can't be mapped in regression models, or we can't calculate its correlation with the Y value without converting it to a numerical value.

For now, we will drop these columns and not use it.
In the next exercise, we will learn how to convert it to something called "One Hot Binary" or to a numerical value manually. You can look up One Hot Binary [here](https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/)

In [7]:
for col_name in df.columns:
    try:
        df[col_name] = df[col_name].astype('float')
    except:
        continue
df.dtypes

symboling            float64
normalized-losses    float64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight          float64
engine-type           object
num-of-cylinders      object
engine-size          float64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower           float64
peak-rpm             float64
city-mpg             float64
highway-mpg          float64
price                float64
dtype: object

In [8]:
for col_name in df.columns:
    if df[col_name].dtype == 'object':
        df = df.drop(col_name, axis='columns')
df.dtypes

symboling            float64
normalized-losses    float64
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight          float64
engine-size          float64
bore                 float64
stroke               float64
compression-ratio    float64
horsepower           float64
peak-rpm             float64
city-mpg             float64
highway-mpg          float64
price                float64
dtype: object

Now, we want to check the correlation between Y and the Xs in the dataset to choose the ones with a strong correlation with Y only

In [9]:
# loop on all X columns, and calc. R_value and store it in an array
Xs = df.iloc[:, 0:-1]
X_names = Xs.columns
r_vals = {}
for x_name in X_names:
    r_val = df[[x_name, 'price']].corr().iloc[0,1]
    print("correlation between", x_name, 'and price is:', r_val)
    r_vals[x_name] = r_val

correlation between symboling and price is: -0.16279428140584462
correlation between normalized-losses and price is: 0.20276129990709132
correlation between wheel-base and price is: 0.7344189369207
correlation between length and price is: 0.7609521757548674
correlation between width and price is: 0.8433705132929789
correlation between height and price is: 0.24483625452119842
correlation between curb-weight and price is: 0.8936390528402931
correlation between engine-size and price is: 0.8414956026114564
correlation between bore and price is: 0.533890352151915
correlation between stroke and price is: 0.16066434143610184
correlation between compression-ratio and price is: 0.20936146850701248
correlation between horsepower and price is: 0.7598739453801003
correlation between peak-rpm and price is: -0.1719160723256923
correlation between city-mpg and price is: -0.6922730619020598
correlation between highway-mpg and price is: -0.7200900979318619


In [15]:
# let's drop columns with r_values in range [-0.5, 0.5] because these don't have a strong correlation with the price
for col_name in r_vals:
    if r_vals[col_name] < 0.5 and r_vals[col_name] > -0.5:
        df = df.drop(col_name, axis='columns', errors='ignore')

In [16]:
df

,wheel-base,length,width,curb-weight,engine-size,bore,horsepower,city-mpg,highway-mpg,price
3,99.8,176.6,66.2,2337.0,109.0,3.19,102.0,24.0,30.0,13950.0
4,99.4,176.6,66.4,2824.0,136.0,3.19,115.0,18.0,22.0,17450.0
6,105.8,192.7,71.4,2844.0,136.0,3.19,110.0,19.0,25.0,17710.0
8,105.8,192.7,71.4,3086.0,131.0,3.13,140.0,17.0,20.0,23875.0
10,101.2,176.8,64.8,2395.0,108.0,3.50,101.0,23.0,29.0,16430.0
...,...,...,...,...,...,...,...,...,...,...
200,109.1,188.8,68.9,2952.0,141.0,3.78,114.0,23.0,28.0,16845.0
201,109.1,188.8,68.8,3049.0,141.0,3.78,160.0,19.0,25.0,19045.0
202,109.1,188.8,68.9,3012.0,173.0,3.58,134.0,18.0,23.0,21485.0
203,109.1,188.8,68.9,3217.0,145.0,3.01,106.0,26.0,27.0,22470.0


## Part Two: Data Division into Train, Validation, and Testing
------------------------------------------------------------------------------------------
We will divide the dataset we have into two partitions.

One is the training .. the part we will train the model on.

And the other is the testing .. this part is used in the end to get the accuracy of the model by testing it on this part of the dataset which the model haven't seen.

We will make these partitions once manually, and once using a built-in function in sklearn called ["import train_test_split"](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [11]:
# we will first define the sizes of each partition as following
train_size = int(len(df) * 0.8)
test_size = len(df) - train_size
print(len(df))
print(train_size, test_size)

159
127 32


In [12]:
# then we will take partition of the dataset depending on these sizes
# we will use iloc function in pandas object df, taking the size we want from the rows and ALL COLUMNS.
# remember iloc is used like this: part = df.iloc[row1:row2, col1:col2]
# and if we didn't write any range, then it takes all
train_df = df.iloc[0 : train_size, ]
test_df = df.iloc[train_size : , ]
print(len(train_df), len(test_df))

127 32


In [17]:
# Now we will seperate the Y from the optional Xs for each of the three partitions
Y_train = train_df['price']
Xs_train = train_df.iloc[:, 0:-1]

Y_test = test_df['price']
Xs_test = test_df.iloc[:, 0:-1]

print(Y_train.shape)
print(Xs_train.shape)

(127,)
(127, 9)


In [18]:
# using sklearn could be done something like this
from sklearn.model_selection import train_test_split

In [19]:
# the function returns two partitions, so we will use it to divide it to train and test

train_df, test_df = train_test_split(df, test_size=0.2)
print(len(train_df), len(test_df))

127 32


In [20]:
# Now we will seperate the Y from the optional Xs for each of the three partitions
Y_train = train_df['price']
# Xs_train = train_df.loc[:, 'symboling':'highway-mpg']
Xs_train = train_df.iloc[:, 0:-1]

Y_test = test_df['price']
Xs_test = test_df.iloc[:, 0:-1]

print(Y_train.shape, Y_test.shape)
print(Xs_train.shape, Xs_test.shape)

(127,) (32,)
(127, 9) (32, 9)


It's better to use the built-in function in sklearn than do it manually because the function also randomize the data
So you don't take the data in sequence, but a random ratio of it

## Part Three: Model Building
-------------------------------------------------
Our multilinear regression model would be as following, considering we have 15 parameters:

Y = a0 + a1 * X1 + a2 * X2 + ... + a15 * X15

In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
regression_model = LinearRegression()
regression_model.fit(Xs_train, Y_train)

LinearRegression()

In [23]:
print('a0 =', regression_model.intercept_)

for i in range(len(Xs_train.columns)):
    print('a', i+1, ' = ', regression_model.coef_[i], '\t-->\t coef. of: ', Xs_train.columns[i], sep='')

a0 = -51960.978393105695
a1 = 110.35005017509455	-->	 coef. of: wheel-base
a2 = -77.7314852199313	-->	 coef. of: length
a3 = 827.6904676888399	-->	 coef. of: width
a4 = 6.672649925049464	-->	 coef. of: curb-weight
a5 = 25.50697341123059	-->	 coef. of: engine-size
a6 = -2140.378035921295	-->	 coef. of: bore
a7 = 14.724833398446636	-->	 coef. of: horsepower
a8 = -10.986265277387703	-->	 coef. of: city-mpg
a9 = -56.5327707754024	-->	 coef. of: highway-mpg


## Part Four: Testing the model & Calculating Accuracy
--------------------------------------------------------------------------------
We will use R^2 metric that is used to measure the accuracy of a model.
It's called the coefficient of determination, the best score is a 1 and the worse is 0.
function [r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) in sklearn measure this right away.

To see how it is calculated, check the video [here](https://www.youtube.com/watch?v=w2FKXOa0HGA).

In [24]:
from sklearn.metrics import r2_score

In [25]:
Y_pred = regression_model.predict(Xs_test)
r2_val = r2_score(Y_test, Y_pred)
print('Accuracy of the model = %.2f' % (r2_val * 100), '%', sep='')

Accuracy of the model = 84.08%


## Part Five: Calculating Bias, Variance & Irreducible Error
-----------------------------------------------------------------------------------------
In this part, we will use a function called [bias_variance_decomp](http://rasbt.github.io/mlxtend/user_guide/evaluate/bias_variance_decomp/) in mlxtend module in python that takes the model and the data, and calculates all three errors right away.

It does this by varying the data multiple times (num_rounds) and calculating which loss is irreducible, and which is varying and which is assumed by the model.

In [26]:
from mlxtend.evaluate import bias_variance_decomp

In [27]:
# Inputs to the function need to be numpy format and not pandas object
Xs_train = Xs_train.to_numpy()
Y_train = Y_train.to_numpy()
Xs_test = Xs_test.to_numpy()
Y_test = Y_test.to_numpy()

In [28]:
total_mse, bias, var = bias_variance_decomp(regression_model, Xs_train, Y_train, Xs_test, Y_test, loss='mse', num_rounds=200, random_seed=1)
# summarize results
print('MSE: %.3f' % total_mse)
print('Bias: %.3f' % bias)
print('Variance: %.3f' % var)
print('Bias + Var.: %.3f' % (bias + var)) # same as total_mse, hence almost no irreducible error .. GOOD!

MSE: 4419332.945
Bias: 3664764.349
Variance: 754568.596
Bias + Var.: 4419332.945


## Part Seven (Extra): Using One Hot Binary
-------------------------------------
In this part, we will convert the non-numeric columns into "one hot binary" columns.

What this does is as follows:

Assume you have a column called color, and all the values in it are one of three values; red, green blue.

When you get the one hot binary encoding of them, you convert this single "color" column, into three columns with names "color_red", "color_green", and "color_blue". Each of these columns has a binary value of 0 or 1.

Now, if in some row the value of color was red, then in the updated columns it would be as: {'color_red': 1, 'color_green': 0, 'color_blue': 0}. And if some column has value blue, then the updated columns would be as: {'color_red': 0, 'color_green': 0, 'color_blue': 1}.

To sum up, what OHB encoding does is converting one column with N unique values into N binary columns, and the value of the original column maps to a '1' value in its column in the updated N columns, and zero in the others.

N.B. This could is a little complicated, it's okay if it is not totally clear. Try print the shapes of the variables before and after each concatenate or hstack functions. Check their documentation. Try yourself. You would understand it more.

As you would see in the following example of the automobile dataset...

In [29]:
new_columns = []
new_columns_names = None

for col_name in df.columns:
    if df[col_name].dtype == 'object':
        OHB = pd.get_dummies(df[col_name], prefix=col_name)
        
        new_columns_names = np.concatenate((new_columns_names, OHB.columns.to_numpy()), axis=None)
        new_columns = np.hstack((new_columns, OHB.to_numpy()))
        
    else:
        if len(new_columns) == 0:
            new_columns = df[col_name].to_numpy().reshape(-1, 1)
            new_columns_names = np.array([col_name])
        else:
            new_columns = np.hstack((new_columns, df[col_name].to_numpy().reshape(-1, 1)))
            new_columns_names = np.concatenate((new_columns_names, col_name), axis=None)

In [30]:
assert new_columns.shape[1] == len(new_columns_names)

In [31]:
print(len(new_columns_names))
print(new_columns_names)

10
['wheel-base' 'length' 'width' 'curb-weight' 'engine-size' 'bore'
 'horsepower' 'city-mpg' 'highway-mpg' 'price']


In [32]:
# converting the new data back into a dataframe:
df = pd.DataFrame(new_columns, columns=new_columns_names)
df

,wheel-base,length,width,curb-weight,engine-size,bore,horsepower,city-mpg,highway-mpg,price
0,99.8,176.6,66.2,2337.0,109.0,3.19,102.0,24.0,30.0,13950.0
1,99.4,176.6,66.4,2824.0,136.0,3.19,115.0,18.0,22.0,17450.0
2,105.8,192.7,71.4,2844.0,136.0,3.19,110.0,19.0,25.0,17710.0
3,105.8,192.7,71.4,3086.0,131.0,3.13,140.0,17.0,20.0,23875.0
4,101.2,176.8,64.8,2395.0,108.0,3.50,101.0,23.0,29.0,16430.0
...,...,...,...,...,...,...,...,...,...,...
154,109.1,188.8,68.9,2952.0,141.0,3.78,114.0,23.0,28.0,16845.0
155,109.1,188.8,68.8,3049.0,141.0,3.78,160.0,19.0,25.0,19045.0
156,109.1,188.8,68.9,3012.0,173.0,3.58,134.0,18.0,23.0,21485.0
157,109.1,188.8,68.9,3217.0,145.0,3.01,106.0,26.0,27.0,22470.0


In [33]:
df.dtypes

wheel-base     float64
length         float64
width          float64
curb-weight    float64
engine-size    float64
bore           float64
horsepower     float64
city-mpg       float64
highway-mpg    float64
price          float64
dtype: object

In [34]:
# loop on all X columns, and calc. R_value and store it in an array
Xs = df.iloc[:, 0:-1]
X_names = Xs.columns
r_vals = {}
for x_name in X_names:
    r_val = df[[x_name, 'price']].corr().iloc[0,1]
    print("correlation between", x_name, 'and price is:', r_val)
    r_vals[x_name] = r_val

correlation between wheel-base and price is: 0.7344189369207
correlation between length and price is: 0.7609521757548674
correlation between width and price is: 0.8433705132929789
correlation between curb-weight and price is: 0.8936390528402931
correlation between engine-size and price is: 0.8414956026114564
correlation between bore and price is: 0.533890352151915
correlation between horsepower and price is: 0.7598739453801003
correlation between city-mpg and price is: -0.6922730619020598
correlation between highway-mpg and price is: -0.7200900979318619


In [35]:
# let's drop columns with r_values in range [-0.5, 0.5] because these don't have a strong correlation with the price
for col_name in r_vals:
    if r_vals[col_name] < 0.5 and r_vals[col_name] > -0.5:
        df = df.drop(col_name, axis='columns')

In [36]:
df

,wheel-base,length,width,curb-weight,engine-size,bore,horsepower,city-mpg,highway-mpg,price
0,99.8,176.6,66.2,2337.0,109.0,3.19,102.0,24.0,30.0,13950.0
1,99.4,176.6,66.4,2824.0,136.0,3.19,115.0,18.0,22.0,17450.0
2,105.8,192.7,71.4,2844.0,136.0,3.19,110.0,19.0,25.0,17710.0
3,105.8,192.7,71.4,3086.0,131.0,3.13,140.0,17.0,20.0,23875.0
4,101.2,176.8,64.8,2395.0,108.0,3.50,101.0,23.0,29.0,16430.0
...,...,...,...,...,...,...,...,...,...,...
154,109.1,188.8,68.9,2952.0,141.0,3.78,114.0,23.0,28.0,16845.0
155,109.1,188.8,68.8,3049.0,141.0,3.78,160.0,19.0,25.0,19045.0
156,109.1,188.8,68.9,3012.0,173.0,3.58,134.0,18.0,23.0,21485.0
157,109.1,188.8,68.9,3217.0,145.0,3.01,106.0,26.0,27.0,22470.0


In [99]:
train_df, test_df = train_test_split(df, test_size=0.2)

Y_train = train_df['price']
Xs_train = train_df.iloc[:, 0:-1]

Y_test = test_df['price']
Xs_test = test_df.iloc[:, 0:-1]

lr_model = LinearRegression()
lr_model = lr_model.fit(Xs_train, Y_train)
Y_pred = lr_model.predict(Xs_test)
r2_val = r2_score(Y_test, Y_pred)
print('Accuracy of the model = %.2f' % (r2_val * 100), '%', sep='')

Accuracy of the model = 91.09%
